In [8]:
import pandas as pd

lots = pd.read_csv("../../web/exports_igp/lots.csv", encoding="iso8859_15", delimiter=";", decimal=",", 
                   dtype={'Code postal Opérateur': 'str', 'Campagne': 'str', 'Num dossier': 'str', 
                          'Num lot': 'str', 'Millésime': 'str'}, low_memory=False)
lots.columns

Index(['Id Opérateur', 'Nom Opérateur', 'Adresse Opérateur',
       'Code postal Opérateur', 'Commune Opérateur', 'Campagne', 'Date lot',
       'Num dossier', 'Num lot', 'Num logement Opérateur', 'Certification',
       'Genre', 'Appellation', 'Mention', 'Lieu', 'Couleur', 'Cepage',
       'Produit', 'Cépages', 'Millésime', 'Spécificités', 'Volume',
       'Statut de lot', 'Destination', 'Date de destination',
       'Pays de destination', 'Elevage', 'Centilisation', 'Date prélévement',
       'Conformité', 'Date de conformité en appel', 'Organisme', 'Doc Id',
       'Lot unique Id', 'Hash produit'],
      dtype='object')

In [52]:
lots_2020 = lots[(lots['Millésime'] == '2020') & (lots['Date lot'] >= '2020-08-01') & (lots['Date lot'] <= '2021-06-15')]
lots_2019 = lots[(lots['Millésime'] == '2019') & (lots['Date lot'] >= '2019-08-01') & (lots['Date lot'] <= '2020-06-15')]

stat_igp = lots_2020.groupby('Produit')[['Volume']].sum().rename(columns={"Volume": "VRT 2020"})
stat_igp['VRT 2019'] = lots_2019.groupby('Produit')[['Volume']].sum()

stat_igp

,VRT 2020,VRT 2019
Produit,,
Alpes Maritimes Blanc,74.64,82.07
Alpes Maritimes Rosé,81.10,90.50
Alpes Maritimes Rouge,203.00,155.62
Alpilles Blanc,3950.61,3229.43
Alpilles Rosé,12703.54,13456.59
...,...,...
Vaucluse Principauté d'Orange Blanc,2334.71,2888.27
Vaucluse Principauté d'Orange Rosé,2612.39,6391.75
Vaucluse Principauté d'Orange Rouge,7252.32,7310.20


In [45]:
historique = pd.read_csv("../../web/exports_igp/lots-historique.csv", encoding="iso8859_15", delimiter=";", decimal=",", 
                   dtype={'Code postal Opérateur': 'str', 'Campagne': 'str', 'Num dossier': 'str', 
                          'Num lot': 'str', 'Millésime': 'str'}, low_memory=False)
lots_controles = historique[(historique['Statut'] == 'Réputé conforme') | (historique['Statut'] == 'Conforme')]
lots_controles.columns

Index(['Id Opérateur', 'Nom Opérateur', 'Campagne', 'Date lot', 'Num Dossier',
       'Num Lot', 'Doc Ordre', 'Doc Type', 'Produit', 'Volume', 'Statut',
       'Details', 'Organisme', 'Doc Id', 'Lot unique Id', 'Hash produit'],
      dtype='object')

In [53]:
lots_controles_2020 = lots_controles[(lots_controles['Campagne'] == '2020-2021') & (lots_controles['Date lot'] >= '2020-08-01') & (lots_controles['Date lot'] <= '2021-06-15')]
lots_controles_2019 = lots_controles[(lots_controles['Campagne'] == '2019-2020') & (lots_controles['Date lot'] >= '2019-08-01') & (lots_controles['Date lot'] <= '2020-06-15')]

lots_controles_2020.groupby('Produit')[['Volume']].sum()

stat_igp['VRC 2020'] = lots_controles_2020.groupby('Produit')[['Volume']].sum()
#stat_igp['VRC 2019'] = lots_controles_2019.groupby('Produit')[['Volume']].sum()
#stat_igp

,Volume
Produit,
Alpes Maritimes Blanc 2020 (N° B20),25.00
Alpes Maritimes Blanc 2020 (N° BLANCI),13.50
Alpes Maritimes Blanc 2020 (N° BLANCII),5.30
Alpes Maritimes Blanc 2020 (N° BLANCIII),4.50
Alpes Maritimes Blanc 2020 (N° FUT4-C6-BOMBONNE1),4.45
...,...
Var Sainte Baume Rouge 2020 (N° 201),650.00
Var Sainte Baume Rouge 2020 (N° B5-B6-B2),17.00
Var Sainte Baume Rouge 2020 (N° F307),30.00
